# Oplossingen Airports

Importeer hier alle nodige bibliotheken:

In [ ]:
# hier komt jouw code
import pandas as pd

Lees de volgende CSV-bestanden met de vertragingsgegevens van Amerikaanse luchtvaartmaatschappijen in in twee DataFrames. Je vindt deze gegevens op volgende URL's:

- https://raw.githubusercontent.com/nickdcox/learn-airline-delays/main/delays_2018.csv (noem dit dataframe `delays2018`)
- https://raw.githubusercontent.com/nickdcox/learn-airline-delays/main/delays_2019.csv (noem dit dataframe `delays2019`)

Je vindt daarin volgende gegevens:

```{text}
            Column Name                                                                                           Description
0                  date                                                   Year and month, in the format YYYY-M (e.g., 2018-1)
1               carrier                                                 The two character designator for the carrier/airline.
2          carrier_name                                                                 The full name of the carrier/airline.
3               airport                                               The three character designator for the arrival airport.
4          airport_name                                                                 The full name of the arrival airport.
5           arr_flights            The total number of arriving flights for the carrier-airport pair for the month specified.
6             arr_del15   The number of arriving flights that were delayed. Delayed is when a flight arrives more than 15 ...
7            carrier_ct                                        The number of arriving flights delayed due to a carrier issue.
8            weather_ct                                        The number of arriving flights delayed due to a weather issue.
9                nas_ct                            The number of arriving flights delayed due to a national air system issue.
10          security_ct                                      The number of arriving flights delayed due to a security issue.
11     late_aircraft_ct                The number of arriving flights delayed due to an earlier late arrival of an aircraft.
12        arr_cancelled                                                                     The number of cancelled flights.
13         arr_diverted                                                                      The number of diverted flights.
14            arr_delay                                                   The total number of delayed minutes due to delays.
15        carrier_delay                                           The total number of delayed minutes due to carrier issues.
16        weather_delay                                           The total number of delayed minutes due to weather issues.
17            nas_delay                               The total number of delayed minutes due to national air system issues.
18       security_delay                                          The total number of delayed minutes due to security issues.
19  late_aircraft_delay                        The total number of delayed minutes due to earlier later arrival of aircraft.
```

In [ ]:
# hier komt jouw code
delays2018 = pd.read_csv("delays_2018.csv", sep=",", decimal=".", header=0)
delays2019 = pd.read_csv("delays_2019.csv", sep=",", decimal=".", header=0)
delays2018.info()

Concateneer de twee dataframes tot één dataframe genaamd `delays` en geef vervolgens het totale aantal rijen weer. Je kan dataframes concateneren met de functie `pd.concat()`. Maak een nieuwe index aan bij het concateneren, dat kan met een bepaalde parameter. In totaal zou je 41177 rijen moeten bekomen. Controleer dat.

In [ ]:
# hier komt jouw code
delays = pd.concat([delays2018, delays2019], ignore_index=True)
print(delays.tail())
print(len(delays))

Welke rijen hebben minstens 1 van volgende eigenschappen?
* de luchthaven heeft geen arriving flights (ontbrekende waarde voor arr_flights)
* lege carrier of lege airports

Verwijder deze rijen uit de tabel
In totaal zou je nu nog 41097 rijen moeten overhouden. Controleer dat.

In [ ]:
# hier komt jouw code
lijnen = delays.arr_flights.isna() | delays.carrier.isna() | delays.airport.isna()
lijn_nummers = lijnen.index[lijnen]
print(lijn_nummers)
delays.drop(lijn_nummers, inplace=True)
len(delays)

Geef een lijst weer van alle luchthavens waarvan de airport in "Tennessee" ligt. Deze luchthavens hebben de letters "TN:" in hun "airport_name" zitten. Dit zouden er 5 moeten zijn.  Je kan dit met een for lus doen, maar er is ook een methode "contains()" die je kan gebruiken op een pandas Series.  Het resultaat is een lijst van booleans die weergeeft op welke plaats het gezochte patroon voorkomt.

In [ ]:
# hier komt jouw code
rijen = delays['airport_name'].str.contains("TN:")
delays['airport'][rijen].unique()

Importeer het coördinatenbestand airport_coordinates.csv. Dit bestand bevat de lengte -en breedtegraden van verschillende Amerikaanse luchthavens evenals hun airport-code

In [ ]:
# hier komt jouw code
airports_coord = pd.read_csv("airport_coordinates.csv", sep=",", decimal=".", header=0)
airports_coord.head()

Volgende code wordt al gegeven.  Ze maakt een nieuw dataframe, genaamd "airports", met een lijst van luchthavens en hun code ("airport" en "airport_name"). In dit nieuwe dataframe worden duplicaten verwijderd. Daarna worden de twee dataframes samengevoegd op de `airport`-sleutel met de "merge"-methode.  Het resultaat is een tabel met "airport", "airport_name", "lat" en "long" als komommen.

In [ ]:
# keep
airports = delays[['airport', 'airport_name']]
airports = airports.drop_duplicates()
result = airports.merge(airports_coord)

Bereken de gemiddelde lengte- en breedtegraad van alle luchthavens waarvan de naam begint met de letter 'B'.  Je kan hierbij de methode "startswith()" van Series gebruiken.

In [ ]:
# hier komt jouw code
meanLat = result[result.airport_name.str.startswith('B')].lat.mean()
print("meanLat=", meanLat)
meanLong = result[result.airport_name.str.startswith('B')].long.mean()
print("leanLong=", meanLong)

Bereken de minimale en maximale waarden voor latitude en longitude.  Waar zijn de luchthavens gesitueerd?

In [ ]:
# hier komt jouw code
minLat = result.lat.min()
print(minLat)
maxLat = result.lat.max()
print(maxLat)
minLong = result.long.min()
print(minLong)
maxLong = result.long.max()
print(maxLong)
# alle luchthavens bevinden zich in de Verenigde Staten

Voeg een nieuwe kolom "zone" toe die aangeeft waar de luchthaven zich ergens bevindt.  Er zijn vier zones:
- "ZW": Zuid-Westelijk (lat < 44.48, long < -120.72)
- "ZO": Zuid-Oostelijk (lat < 44.48, long > -120.72)
- "NW": Noord-Westelijk (lat > 44.48, long < -120.72)
- "NO": Noord-Oostelijk (lat > 44.48, long > -120.72)
Gebruik de afkortingen ZW, ZO, NW en NO als waarden voor de nieuwe kolom.


In [ ]:
# hier komt jouw code
kolom = []
for rij in range(len(result)):
    waarde = "NO"
    if result.lat[rij]<44.48 and result.long[rij] < -120.72:
        waarde = "ZW"
    if result.lat[rij]<44.48 and result.long[rij] > -120.72:
        waarde = "ZO"
    if result.lat[rij]>44.48 and result.long[rij] < -120.72:
        waarde = "NW"
    kolom = kolom + [ waarde ]
result["zone"] = kolom
result

Bereken nu de gemiddelde latitude en longitude per zone.  Je mag dit doen met een for loop.  Er is een methode "mean()" die het gemiddelde van een Series berekent.  Die mag je gebruiken.

In [ ]:
# hier komt jouw code
for zone in result.zone.unique():
    lat = result.lat[result.zone == zone]
    gem_lat = lat.mean()
    long = result.long[result.zone == zone]
    gem_long = long.mean()
    print(zone, ": ", gem_lat, ", ", gem_long)